In [14]:
##########################################
## Get the File Sizes
##########################################
import pandas as pd
import os
from pyspark.sql.functions import col, year, month, quarter, to_date, dayofmonth,current_timestamp
import time
import warnings
from pyspark.sql import SparkSession
from datetime import datetime,date,timedelta

# Reference https://stackoverflow.com/questions/61547289/loop-to-pull-file-size-folder-size-and-directory-size

# Files Location
dir_path = '/lakehouse/default/Files'

# Collect data for all files in the directory
data = {'directory': [], 'file_name': [], 'file_size': [], 'file_date': []}
for dirpath, dirnames, filenames in os.walk(dir_path):
    for f in filenames:
        filename = "{}\{}" .format(dirpath, f)
        data["directory"].append(dirpath)
        data["file_name"].append(f)
        data["file_size"].append(os.path.getsize(filename))
        data["file_date"].append(time.strftime('%m/%d/%Y', time.gmtime(os.path.getmtime(filename))))

# Transform data in dataframe
files = pd.DataFrame(data)

## Write to Lakehouse Table
filename = 'FileSizes_' + (datetime.today()).strftime('%Y%m%d') + '.csv'
files.to_csv("/lakehouse/default/Files/" + filename)
spk_items = spark.createDataFrame(files)
# Adding Current Date Time to Lakehouse Table
df_with_datetime = spk_items.withColumn("CurrentDateTime", current_timestamp())
df_with_datetime.write.mode("append").format("delta").option("overwriteSchema", "true").saveAsTable('FileSizes')

StatementMeta(, a435fb16-5f0a-490b-b068-ef7778ccad28, 22, Finished, Available)

/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.


In [4]:
# Install CAT TOOLS
%pip install "https://raw.githubusercontent.com/m-kovalsky/fabric_cat_tools/main/fabric_cat_tools-0.3.0-py3-none-any.whl" --q


StatementMeta(, c5c1f70d-7355-4398-9d60-067667029ded, 11, Finished, Available)


[notice] A new release of pip is available: 23.0 -> 24.0
[notice] To update, run: /nfs4/pyenv-fb96bc74-6f89-4dbe-9e9f-6d98e75715e1/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [5]:
##########################################
# Get Table Data
##########################################
import fabric_cat_tools as fct

# Get Table Details
# Reference: https://github.com/m-kovalsky/fabric_cat_tools?tab=readme-ov-file#get_lakehouse_tables

# Put in the Lakehouse Name
lakehouse_name = 'LH_NYCTaxi'

# Get the Lakehouse Table sizes into Pandas Dataframe
df_tableSizes = fct.get_lakehouse_tables(
        # lakehouse = lakehouse_name
        #,workspace = ''
        extended = True)

StatementMeta(, c5c1f70d-7355-4398-9d60-067667029ded, 13, Finished, Available)

In [17]:
##########################################
# Insert Tables into Lakehouse Table
##########################################
from pyspark.sql.functions import current_timestamp
from datetime import datetime,date,timedelta

## Remove Invalid Characters from Column Names
df_tableSizes.columns = df_tableSizes.columns.str.replace(r'(.*\[)|(\].*)', '', regex=True)
df_tableSizes.columns = df_tableSizes.columns.str.replace(' ', '_')
df_tableSizes.columns = df_tableSizes.columns.str.replace('(', '')
df_tableSizes.columns = df_tableSizes.columns.str.replace(')', '')

## Write to Lakehouse Table
filename = 'Lakehouse_TableSizes_' + (datetime.today()).strftime('%Y%m%d') + '.csv'
df_tableSizes.to_csv("/lakehouse/default/Files/" + filename)
spk_tableSizes = spark.createDataFrame(df_tableSizes)
# Adding Current Date Time to Lakehouse Table
sparkdf_with_datetime = spk_tableSizes.withColumn("CurrentDateTime", current_timestamp())
sparkdf_with_datetime.write.mode("append").format("delta").option("overwriteSchema", "true").saveAsTable('Lakehouse_TableSizes')


StatementMeta(, a435fb16-5f0a-490b-b068-ef7778ccad28, 25, Finished, Available)

/tmp/ipykernel_12628/3266040530.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_tableSizes.columns = df_tableSizes.columns.str.replace('(', '')
/tmp/ipykernel_12628/3266040530.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_tableSizes.columns = df_tableSizes.columns.str.replace(')', '')
/opt/spark/python/lib/pyspark.zip/pyspark/sql/pandas/conversion.py:604: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
